In [2]:
%pip install ipywidgets
%pip install datasets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
# !pip install ipywidgets
# !pip install datasets
from datasets import load_dataset

dataset = load_dataset("imagefolder", data_dir="./cats_vs_dogs")
dataset = dataset["train"]

dataset = dataset.shuffle(seed=1234).select(range(1000)) # 🤔 สลับ random แล้วสุ่มเลือกมาตาม range
dataset = dataset.train_test_split(test_size=0.3, seed=1234) # 🤔
dataset

Resolving data files:   0%|          | 0/1877 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 700
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 300
    })
})

In [14]:
dataset["train"][0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=360x288>,
 'label': 1}

label class

In [16]:
labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()

for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

id2label

{'0': 'cat', '1': 'dog'}

library ของ huggingface อาจจะ

In [18]:
%pip install torch torchvision torchaudio
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

%pip install transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
%pip uninstall torch torchvision torchaudio
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

# !pip install transformers
from transformers import AutoImageProcessor

checkpoint = "microsoft/resnet-50" # 🤔ปรับขนาด รูปภาพ
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

In [7]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor # 🤔 crop normolize

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = image_processor.size["shortest_edge"] if "shortest_edge" in image_processor.size else (image_processor.size["height"], image_processor.size["width"])
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize]) # 🤔

def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

dataset = dataset.with_transform(transforms)
dataset

OSError: [WinError 126] The specified module could not be found. Error loading "d:\Mook\262_ML\lab\.venv\lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [8]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()
data_collator

OSError: [WinError 126] The specified module could not be found. Error loading "d:\Mook\262_ML\lab\.venv\lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [ ]:
# !pip install evaluate
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
# !pip install accelerate
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)

training_args = TrainingArguments(
    output_dir="my_image_model", # 🤔
    remove_unused_columns=False,
    eval_strategy="epoch",
    save_strategy="epoch", # 🤔
    learning_rate=1e-5, # 🤔
    per_device_train_batch_size=1, # 🤔
    gradient_accumulation_steps=1, # 🤔
    per_device_eval_batch_size=1, # 🤔
    num_train_epochs=1, # 🤔
    warmup_ratio=0.1, # 🤔
    logging_steps=10, # 🤔
    load_best_model_at_end=True, # 🤔
    metric_for_best_model="accuracy",
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model("my_final_image_model") # 🤔

In [ ]:
dataset = load_dataset("imagefolder", data_dir="./cats_vs_dogs")
dataset

In [ ]:
image = dataset["train"]["image"][1] # 🤔
image

In [ ]:
from transformers import pipeline

classifier = pipeline("image-classification", model="./my_final_image_model")
classifier(image)